In [1]:
import os
import numpy as np
import pandas as pd
import logging
from shapely.geometry import Point
import plotly.express as px
import geopandas as gpd
import plotly.express as px
from multiprocessing import Pool
from scipy.stats import zscore
from GPSOdyssey import Polaris, Kepler, Void, Vega
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Data

* Good RFID candidates
* RFID merge

### Time shift summary

* **RFID**: +08:00 (By default) | I read with +00:00 time shift
* **GPS**: +06:00 ()

In [10]:
def generate_trip_id(gps):
    gps['trip_id'] = gps['date'].astype('str') + ' ' + gps['vehicle_id']
    return gps

def generate_point_id(gps):
    gps['point_id'] = gps['datetime'].astype('str') + ' ' + gps['vehicle_id']
    return gps

### GPS records

In [11]:
S_CLUSTERED_GPS = '/Data/Outputs/GPS_Clusters/'


vega = Vega(engine='pandas')
gps = vega.read_from_dir(directory=S_CLUSTERED_GPS,
                             file_extensions='.csv',
                             args={'parse_dates': ['time']},
                             concatenate=True, amt_in_parallel=6)
    

gps = Polaris(gps) \
    .select_columns(['truck_id', 'lon', 'lat', 'time', 'lon_match', 'lat_match', 'cluster_id']) \
    .pandas_df_operation(func_name='rename', arguments={'columns': {'truck_id': 'vehicle_id', 'time': 'datetime'}}) \
    .construct_datetime(datetime='datetime', offset='-02:00') \
    .remove_tz(time_col='datetime') \
    .add_date_col('datetime', 'date') \
    .add_time_col('datetime', 'time') \
    .add_unixtime('datetime', 'unixtime') \
    .df

gps = generate_trip_id(gps)
gps = generate_point_id(gps)
gps['unixtime'] = gps['unixtime'].astype('int')

### RFID merge

In [12]:
S_RFID_OCTOBER = '/Data/Source/RFID/RFID_october.csv'


rfid = pd.read_csv(S_RFID_OCTOBER)
rfid['is_rfid_active'] = True

RFID_COLUMNS = ['date', 'time', 'unixtime', 'vehicle_id',
                'RFID', 'is_rfid_active', 'Latitude', 'Longitude']
rfid = rfid.loc[:, RFID_COLUMNS]
rfid.columns = [str(x).lower() for x in rfid.columns]

rfid = rfid.sort_values(by='unixtime')
rfid['datetime'] = rfid['date'] + ' ' + rfid['time']


rfid = generate_trip_id(rfid)
rfid = generate_point_id(rfid)

# Select only suitalbe gps
rfid = rfid.merge(gps[~gps['trip_id'].duplicated(keep='last')][['trip_id']], how='inner', on=['trip_id'])

## Merge: RFID & GPS

In [13]:
rfid['datetime'] = rfid['datetime'].astype('str')
gps['datetime'] = gps['datetime'].astype('str')

### core matching operation

In [14]:
gps_m = gps.merge(rfid[['rfid', 'vehicle_id', 'datetime', 'unixtime',
                        'is_rfid_active', 'latitude', 'longitude']],
                  on=['vehicle_id', 'datetime'], how='outer')

gps_m['is_overlap'] = gps_m['lat_match'].isna()
gps_m.sort_values(['vehicle_id', 'datetime'], inplace=True)

cols2fill = ['lon', 'lat', 'time', 'vehicle_id', 'date', 'datetime', 'time',
             'unixtime_x', 'lon_match', 'lat_match', 'trip_id', 'point_id']
gps_m[cols2fill] = gps_m[cols2fill].fillna(method='ffill')

gps_m['is_rfid_active'] = ~gps_m['rfid'].isna()

In [15]:
gps_m[['longitude', 'latitude']] = gps_m[['longitude', 'latitude']].astype('float')

In [16]:
MAP_CONFIG = 'keplerConf/gps_rfid_match_viz.json'

kepler = Kepler({'gps': gps_m, 'rfid': rfid},
                height=800, config_path=MAP_CONFIG)
kepler.render_kepler_map()
kepler.get_rendered_map()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['gps'], 'id': '2tn8qm4hv', '…